# Other Artist Lyric Data Collection and Cleaning

50 songs of 6 other artists for classification purposes

In [1]:
import requests
import json
import time
import lyricsgenius as lg


In [2]:
# Set up API access
GENIUS_API_TOKEN = "YIHKEmRaW8tuw7qpsPhPf3_BZCEIp9N53yr2BsNOWi_qvjCA-WVskFIK7hL-tW6Q"
BASE_URL = "https://api.genius.com"
headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
artists = ['Linkin Park', 'Kanye West', 'Coldplay', 'Beyoncé', 'Carrie Underwood', 'Rihanna']


In [3]:
genius = lg.Genius(GENIUS_API_TOKEN, skip_non_songs=True, excluded_terms=["(Remix)", "(Live)"], remove_section_headers=True)

## Functions to process song lyrics

In [23]:
from string import printable
import re

## Function to take a string and clean it of non-printable characters
def clean_str(input_str):
  output_str = re.sub(r'[^\x00-\x7f]', r'', input_str)
  return output_str.strip()


In [24]:
def clean_song(song, unique_titles):
  title = clean_str(song.title)

  #Check if the song is a re-recorded version/Remove non-song entries (speeches, dates, guide, costumes)

  if (title in unique_titles):
    print("Repeated title")
    return {}

  if(title.find("(") != -1):
    idx = title.find("(")
    root_title = title[:idx].strip()

    if (root_title in unique_titles):
      print("Repeated title")
      return {}
    else:
      title = root_title

  try:
    junk, l = song.lyrics.split('\n', 1)

  except:
    print("No junk")
    l = song.lyrics

  finally:
    lyrics = clean_str(l)

    d = {'title': title,
         'lyrics': lyrics}

    unique_titles.append(title)

    return d


In [25]:
from requests.exceptions import HTTPError, Timeout
genius.timeout = 8

def get_artist (artist_name):
  try:
    artist = genius.search_artist(artist_name, max_songs=50)
  except HTTPError as e:
      print(e.errno)    # status code
      print(e.args[0])  # status code
      print(e.args[1])  # error message
  except Timeout:
      pass
  finally:
    artist.save_lyrics()
    return artist


### Collect data for each artist

In [26]:
#Number of valid songs
song_ct = 0

# Dictionary to store contents of json file
data = {}
data['songs'] = []

for artist_name in artists:
  print("-----------\n", artist_name)
  artist=get_artist(artist_name)
  unique_titles = []
  
  for i, song in enumerate(artist.songs):
    # print(song.title)
    # print("Song #", i)
    d = clean_song(song, unique_titles)
    if (d == {}):
      continue
    data['songs'].append(d)
    song_ct += 1


-----------
 Linkin Park
Searching for songs by Linkin Park...

Song 1: "In the End"
Song 2: "One More Light"
Song 3: "Heavy"
Song 4: "Numb"
Song 5: "Crawling"
Song 6: "Battle Symphony"
Song 7: "Breaking the Habit"
Song 8: "What I’ve Done"
Song 9: "Good Goodbye"
Song 10: "Somewhere I Belong"
Song 11: "CASTLE OF GLASS"
Song 12: "Faint"
Song 13: "Talking to Myself"
Song 14: "Nobody Can Save Me"
Song 15: "Papercut"
Song 16: "Leave Out All the Rest"
Song 17: "Bleed It Out"
Song 18: "Waiting for the End"
Song 19: "One Step Closer"
Song 20: "Lost"
Song 21: "BURN IT DOWN"
Song 22: "A Place for My Head"
Song 23: "LOST IN THE ECHO"
Song 24: "New Divide"
Song 25: "Shadow of the Day"
Song 26: "Invisible"
Song 27: "Given Up"
Song 28: "Sharp Edges"
Song 29: "Points of Authority"
Song 30: "ROADS UNTRAVELED"
Song 31: "By Myself"
Song 32: "From the Inside"
Song 33: "Iridescent"
Song 34: "Lying from You"
Song 35: "Sorry for Now"
Song 36: "The Catalyst"
Song 37: "Hands Held High"
Song 38: "With You"
Son

In [28]:
song_ct

300

### Writing data into a JSON file

In [27]:
# Writing into a json file

import json

json_object = json.dumps(data, indent=2)

# Writing to sample.json
with open("other_data.json", "w") as outfile:
    outfile.write(json_object)
